# Polarization example - Maximum likelihood method

This notebook fits the polarization fraction and angle of a Data Challenge 3 GRB (GRB 080802386) simulated using MEGAlib and combined with albedo photon background. It's assumed that the start time, duration, localization, and spectrum of the GRB are already known. The GRB was simulated with 80% polarization at an angle of 90 degrees in the IAU convention, and was 20 degrees off-axis. 

In [11]:
from cosipy import UnBinnedData, BinnedData, COSILike
from cosipy.spacecraftfile import SpacecraftFile
from cosipy.polarization.conventions import MEGAlibRelativeX, MEGAlibRelativeY, MEGAlibRelativeZ, IAUPolarizationConvention
from cosipy.threeml.custom_functions import Band_Eflux
from astropy.time import Time
import numpy as np
from astropy.coordinates import Angle, SkyCoord
from astropy import units as u
from cosipy.util import fetch_wasabi_file
from threeML import LinearPolarization, SpectralComponent, PointSource, Model, JointLikelihood, DataList
from astromodels import Parameter

### Read in data

In [ ]:
grb_background_local = UnBinnedData('grb.yaml')
grb_background_local.select_data_time(unbinned_data='/Users/eneights/work/COSI/data-challenges/data_challenge_4/data/simulations/polarization_tutorial_grb_background.fits.gz', output_name='grb_background_source_interval')
grb_background_local = BinnedData('grb.yaml')
grb_background_local.get_binned_data(unbinned_data='grb_background_source_interval.fits.gz', output_name='grb_background_binned_local', psichi_binning='local')
grb_background_local.load_binned_data_from_hdf5('grb_background_binned_local.hdf5')

background_before_local = UnBinnedData('background_before.yaml')
background_before_local.select_data_time(unbinned_data='/Users/eneights/work/COSI/data-challenges/data_challenge_4/data/simulations/polarization_tutorial_grb_background.fits.gz', output_name='background_before')
background_before_local = BinnedData('background_before.yaml')
background_before_local.get_binned_data(unbinned_data='background_before.fits.gz', output_name='background_before_binned_local', psichi_binning='local')
background_before_local.load_binned_data_from_hdf5('background_before_binned_local.hdf5')

background_after_local = UnBinnedData('background_after.yaml')
background_after_local.select_data_time(unbinned_data='/Users/eneights/work/COSI/data-challenges/data_challenge_4/data/simulations/polarization_tutorial_grb_background.fits.gz', output_name='background_after')
background_after_local = BinnedData('background_after.yaml')
background_after_local.get_binned_data(unbinned_data='background_after.fits.gz', output_name='background_after_binned_local', psichi_binning='local')
background_after_local.load_binned_data_from_hdf5('background_after_binned_local.hdf5')

grb_background_galactic = BinnedData('grb.yaml')
grb_background_galactic.get_binned_data(unbinned_data='grb_background_source_interval.fits.gz', output_name='grb_background_binned_galactic', psichi_binning='galactic')
grb_background_galactic.load_binned_data_from_hdf5('grb_background_binned_galactic.hdf5')

background_before_galactic = BinnedData('background_before.yaml')
background_before_galactic.get_binned_data(unbinned_data='background_before.fits.gz', output_name='background_before_binned_galactic', psichi_binning='galactic')
background_before_galactic.load_binned_data_from_hdf5('background_before_binned_galactic.hdf5')

background_after_galactic = BinnedData('background_after.yaml')
background_after_galactic.get_binned_data(unbinned_data='background_after.fits.gz', output_name='background_after_binned_galactic', psichi_binning='galactic')
background_after_galactic.load_binned_data_from_hdf5('background_after_binned_galactic.hdf5')

Define the path to the detector response and read in the orientation file. The orientation is cut down to the time interval of the source.

In [8]:
response_file = '/Users/eneights/work/COSI/data-challenges/data_challenge_4/data/responses/ResponseContinuum.o3.pol.e200_10000.b4.p12.s10396905069491.m441.filtered.nonsparse.binnedpolarization.11D_nside8.area.h5'

sc_orientation = SpacecraftFile.parse_from_file('/Users/eneights/work/COSI/data-challenges/data_challenge_4/data/orientations/DC3.ori')
sc_orientation = sc_orientation.source_interval(Time(1835493492.2, format = 'unix'), Time(1835493492.8, format = 'unix'))

Define the GRB position and spectrum.

In [ ]:
source_direction = SkyCoord(l=23.53, b=-53.44, frame='galactic', unit=u.deg)
attitude = sc_orientation.get_attitude()[0]

a = 100. * u.keV
b = 10000. * u.keV
alpha = -0.7368949
beta = -2.095031
ebreak = 622.389 * u.keV
K = 300. / u.cm / u.cm / u.s

spectrum = Band_Eflux(a = a.value,
                      b = b.value,
                      alpha = alpha,
                      beta = beta,
                      E0 = ebreak.value,
                      K = K.value)

spectrum.a.unit = a.unit
spectrum.b.unit = b.unit
spectrum.E0.unit = ebreak.unit
spectrum.K.unit = K.unit

polarization = LinearPolarization(80, 80) # polarization degree, polarization angle (in same convention as response for data in local frame and IAU convention for data in galactic frame)
spectral_component = SpectralComponent('grb', spectrum, polarization)

source = PointSource('source',                                                          # Name of source (arbitrary, but needs to be unique)
                     l = source_direction.l.deg,               # Longitude (deg)
                     b = source_direction.b.deg,               # Latitude (deg)
                     components = [spectral_component])                                 # Spectral model

source.components['grb'].shape.K.fix = True
source.components['grb'].shape.E0.fix = True
source.components['grb'].shape.alpha.fix = True
source.components['grb'].shape.beta.fix = True

model = Model(source)

In [29]:
bkg_par = Parameter("background_cosi",                                                                          # background parameter
                    0.0016,                                                                                        # initial value of parameter
                    min_value=0,                                                                                # minimum value of parameter
                    max_value=1,                                                                                # maximum value of parameter
                    delta=1e-5,                                                                                 # initial step used by fitting engine
                    desc="Background parameter for cosi")

cosi = COSILike("cosi",                                                                                         # COSI 3ML plugin
                dr = response_file,                                                                                        # detector response
                data = grb_background_local.binned_data.project('Em', 'Phi', 'PsiChi'),                                      # data (source+background)
                bkg = (background_before_local.binned_data.project('Em', 'Phi', 'PsiChi') + background_after_local.binned_data.project('Em', 'Phi', 'PsiChi')) * 0.0016,    # background model 
                sc_orientation = sc_orientation,
                response_pa_convention = 'RelativeZ')
                #nuisance_param = bkg_par)

plugins = DataList(cosi)

like = JointLikelihood(model, plugins, verbose = False)

The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!


09:27:34 INFO      set the minimizer to minuit                                             ]8;id=740192;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/classicMLE/joint_likelihood.py\joint_likelihood.py]8;;\:]8;id=359590;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/classicMLE/joint_likelihood.py#1045\1045]8;;\

### Polarization fit in local frame

In [30]:
like.fit()

The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
Adding 1e-12 to each bin of the expectation to avoid log-likelihood = -inf.
The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
The response must have the same polarization convention as the provided polarization angle, or the result will be incorrect!
The response must have the same polarization conv

09:30:07 WARNING   get_number_of_data_points not implemented, values for statistical        ]8;id=970920;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/plugin_prototype.py\plugin_prototype.py]8;;\:]8;id=655372;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/plugin_prototype.py#130\130]8;;\
                  measurements such as AIC or BIC are unreliable                                                   

Best fit values:

,result,unit
parameter,,
source.spectrum.grb.polarization.degree,(6.0 +/- 0.6) x 10,
source.spectrum.grb.polarization.angle,(8.32 +/- 0.34) x 10,deg


Correlation matrix:

...

Values of -log(likelihood) at the minimum:

,-log(likelihood)
cosi,18747.564101
total,18747.564101


Values of statistical measures:

,statistical measures
AIC,37493.128202
BIC,37495.128202


(                                             value  negative_error  \
 source.spectrum.grb.polarization.degree  60.304226       -6.271626   
 source.spectrum.grb.polarization.angle   83.164307       -3.372013   
 
                                          positive_error     error unit  
 source.spectrum.grb.polarization.degree        6.140856  6.206241       
 source.spectrum.grb.polarization.angle         3.432037  3.402025  deg  ,
        -log(likelihood)
 cosi       18747.564101
 total      18747.564101)

In [32]:
spectrum = Band_Eflux(a = a.value,
                      b = b.value,
                      alpha = alpha,
                      beta = beta,
                      E0 = ebreak.value,
                      K = K.value)

spectrum.a.unit = a.unit
spectrum.b.unit = b.unit
spectrum.E0.unit = ebreak.unit
spectrum.K.unit = K.unit

polarization = LinearPolarization(80, 90) # convention of angle?? should be input into cosilike somewhere -- same as response for local frame, iau for galactic frame
spectral_component = SpectralComponent('grb', spectrum, polarization)

source = PointSource('source',                                                          # Name of source (arbitrary, but needs to be unique)
                     l = source_direction.l.deg,               # Longitude (deg)
                     b = source_direction.b.deg,               # Latitude (deg)
                     components = [spectral_component])                                 # Spectral model

source.components['grb'].shape.K.fix = True
source.components['grb'].shape.E0.fix = True
source.components['grb'].shape.alpha.fix = True
source.components['grb'].shape.beta.fix = True

model = Model(source)

### Polarization fit in ICRS coordinates

In [33]:
cosi_galactic = COSILike("cosi",                                                                                         # COSI 3ML plugin
                dr = response_file,                                                                                        # detector response
                data = grb_background_galactic.binned_data.project('Em', 'Phi', 'PsiChi'),                                      # data (source+background)
                bkg = (background_before_galactic.binned_data.project('Em', 'Phi', 'PsiChi') + background_after_galactic.binned_data.project('Em', 'Phi', 'PsiChi')) * 0.0016,    # background model 
                sc_orientation = sc_orientation,
                response_pa_convention = 'RelativeZ')
                #nuisance_param = bkg_par)

plugins_galactic = DataList(cosi_galactic)

like_galactic = JointLikelihood(model, plugins_galactic, verbose = False)

like_galactic.fit()

11:11:02 INFO      set the minimizer to minuit                                             ]8;id=48516;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/classicMLE/joint_likelihood.py\joint_likelihood.py]8;;\:]8;id=396566;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/classicMLE/joint_likelihood.py#1045\1045]8;;\

Adding 1e-12 to each bin of the expectation to avoid log-likelihood = -inf.


12:25:23 ERROR     Last status:                                                             ]8;id=182087;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=867331;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#166\166]8;;\

         ERROR     ┌─────────────────────────────────────────────────────────────────────── ]8;id=463832;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=652850;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#169\169]8;;\
                  ──┐                                                                                              

         ERROR     │                                Migrad                                  ]8;id=803436;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=532038;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#169\169]8;;\
                  │                                                                                                

...

         ERROR     ├───┼─────────────────────────────────────────┼───────────┼───────────┼─ ]8;id=268308;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=455215;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#174\174]8;;\
                  ───────────┼────────────┼─────────┼─────────┼───────┤                                            

         ERROR     │ 0 │ source_spectrum_grb_polarization_degree │   29.4    │    3.0    │  ]8;id=423661;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=423095;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#174\174]8;;\
                  │            │    0    │   100   │       │                                                       

         ERROR     │ 1 │ source_spectrum_grb_polarization_angle  │  89.9979  │  0.0014   │  ]8;id=454956;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=360136;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#174\174]8;;\
                  │            │    0    │   180   │       │                                                       

         ERROR     └───┴─────────────────────────────────────────┴───────────┴───────────┴─ ]8;id=302610;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py\minuit_minimizer.py]8;;\:]8;id=204486;file:///Users/eneights/software/miniforge3/envs/cosipy-fork/lib/python3.10/site-packages/threeML/minimizer/minuit_minimizer.py#174\174]8;;\
                  ───────────┴────────────┴─────────┴─────────┴───────┘                                            

FitFailed: MIGRAD call failed. This is usually due to unconstrained parameters.